In [1]:
import pandas as pd
import numpy as np

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data=pd.read_csv('attribution_allocation_student_data.csv')

In [3]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3
68120,False,referral,referral,referral,referral,direct,3
68121,False,display,referral,social,direct,NaN,3
68122,False,email,email,social,direct,NaN,3


In [4]:
data['convert_TF'].value_counts()

False    54345
True     13779
Name: convert_TF, dtype: int64

# Part 1: Attribution

In [5]:
data["convert_TF"] = data["convert_TF"].astype(int)

In [6]:
cols=['touch1','touch2','touch3','touch4','touch5']

In [7]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,1,referral,referral,referral,email,NaN,1
1,1,referral,display,display,email,NaN,1
2,1,email,display,referral,NaN,NaN,1
3,1,referral,referral,email,NaN,NaN,1
4,1,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,0,social,display,direct,NaN,NaN,3
68120,0,referral,referral,referral,referral,direct,3
68121,0,display,referral,social,direct,NaN,3
68122,0,email,email,social,direct,NaN,3


In [8]:
data['path']=data[cols].apply(lambda x: ' > '.join(x.dropna()), axis=1)

In [9]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,path
0,1,referral,referral,referral,email,NaN,1,referral > referral > referral > email
1,1,referral,display,display,email,NaN,1,referral > display > display > email
2,1,email,display,referral,NaN,NaN,1,email > display > referral
3,1,referral,referral,email,NaN,NaN,1,referral > referral > email
4,1,social,referral,NaN,NaN,NaN,1,social > referral
...,...,...,...,...,...,...,...,...
68119,0,social,display,direct,NaN,NaN,3,social > display > direct
68120,0,referral,referral,referral,referral,direct,3,referral > referral > referral > referral > di...
68121,0,display,referral,social,direct,NaN,3,display > referral > social > direct
68122,0,email,email,social,direct,NaN,3,email > email > social > direct


### 1.1 first and last interaction

In [10]:
data['first interaction']=data['path'].map(lambda x: x.split(' > ')[0])

In [11]:
data['last interaction']=data['path'].map(lambda x: x.split(' > ')[-1])

In [12]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,path,first interaction,last interaction
0,1,referral,referral,referral,email,NaN,1,referral > referral > referral > email,referral,email
1,1,referral,display,display,email,NaN,1,referral > display > display > email,referral,email
2,1,email,display,referral,NaN,NaN,1,email > display > referral,email,referral
3,1,referral,referral,email,NaN,NaN,1,referral > referral > email,referral,email
4,1,social,referral,NaN,NaN,NaN,1,social > referral,social,referral
...,...,...,...,...,...,...,...,...,...,...
68119,0,social,display,direct,NaN,NaN,3,social > display > direct,social,direct
68120,0,referral,referral,referral,referral,direct,3,referral > referral > referral > referral > di...,referral,direct
68121,0,display,referral,social,direct,NaN,3,display > referral > social > direct,display,direct
68122,0,email,email,social,direct,NaN,3,email > email > social > direct,email,direct


In [13]:
df_first=pd.DataFrame()
df_first['Channel']=data['first interaction']
df_first['Attribution']='First interaction'
df_first['Conversion']=data['convert_TF']
df_first['tier']=data['tier']
df_first

,Channel,Attribution,Conversion,tier
0,referral,First interaction,1,1
1,referral,First interaction,1,1
2,email,First interaction,1,1
3,referral,First interaction,1,1
4,social,First interaction,1,1
...,...,...,...,...
68119,social,First interaction,0,3
68120,referral,First interaction,0,3
68121,display,First interaction,0,3
68122,email,First interaction,0,3


In [29]:
df_first=df_first.groupby(['Channel']).sum()

In [30]:
df_first

,Conversion
Channel,
direct,6
display,2031
email,1284
organic_search,15
paid_search,668
referral,7495
social,2280


### 1.2 average CAC by channel (first interaction)

In [81]:
first_cac=[['avg_CAC_first_direct',0/6],
['avg_CAC_first_display',6000/2031],
['avg_CAC_first_email',6000/1284],
['avg_CAC_first_organic_search',0/15],
['avg_CAC_first_paid_search',6000/668],
['avg_CAC_first_referral',6000/7495],
['avg_CAC_first_social',6000/2280]]
first_CAC=pd.DataFrame(first_cac, columns=['Channel','CAC'])
first_CAC

,Channel,CAC
0,avg_CAC_first_direct,0.000000
1,avg_CAC_first_display,2.954210
2,avg_CAC_first_email,4.672897
3,avg_CAC_first_organic_search,0.000000
4,avg_CAC_first_paid_search,8.982036
5,avg_CAC_first_referral,0.800534
6,avg_CAC_first_social,2.631579


In [17]:
df_last=pd.DataFrame()
df_last['Channel']=data['last interaction']
df_last['Attribution']='last interaction'
df_last['Conversion']=data['convert_TF']
df_last

,Channel,Attribution,Conversion,tier
0,email,last interaction,1,1
1,email,last interaction,1,1
2,referral,last interaction,1,1
3,email,last interaction,1,1
4,referral,last interaction,1,1
...,...,...,...,...
68119,direct,last interaction,0,3
68120,direct,last interaction,0,3
68121,direct,last interaction,0,3
68122,direct,last interaction,0,3


In [31]:
df_last=df_last.groupby(['Channel']).sum()

In [32]:
df_last

,Conversion
Channel,
direct,7
display,2292
email,1491
organic_search,11
paid_search,1004
referral,6014
social,2960


### 1.3 average CAC by channel (last interaction)

In [82]:
last_cac=[['avg_CAC_last_direct',0/7],
['avg_CAC_last_display',6000/2292],
['avg_CAC_last_email',6000/1491],
['avg_CAC_last_organic_search',0/11],
['avg_CAC_last_paid_search',6000/1004],
['avg_CAC_last_referral',6000/6014],
['avg_CAC_last_social',6000/2960]]
last_CAC=pd.DataFrame(last_cac, columns=['Channel','CAC'])
last_CAC

,Channel,CAC
0,avg_CAC_last_direct,0.000000
1,avg_CAC_last_display,2.617801
2,avg_CAC_last_email,4.024145
3,avg_CAC_last_organic_search,0.000000
4,avg_CAC_last_paid_search,5.976096
5,avg_CAC_last_referral,0.997672
6,avg_CAC_last_social,2.027027


### 1.4 average CAC by channel (linear attributio )

In [23]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,path,first interaction,last interaction
0,1,referral,referral,referral,email,NaN,1,referral > referral > referral > email,referral,email
1,1,referral,display,display,email,NaN,1,referral > display > display > email,referral,email
2,1,email,display,referral,NaN,NaN,1,email > display > referral,email,referral
3,1,referral,referral,email,NaN,NaN,1,referral > referral > email,referral,email
4,1,social,referral,NaN,NaN,NaN,1,social > referral,social,referral
...,...,...,...,...,...,...,...,...,...,...
68119,0,social,display,direct,NaN,NaN,3,social > display > direct,social,direct
68120,0,referral,referral,referral,referral,direct,3,referral > referral > referral > referral > di...,referral,direct
68121,0,display,referral,social,direct,NaN,3,display > referral > social > direct,display,direct
68122,0,email,email,social,direct,NaN,3,email > email > social > direct,email,direct


In [20]:
data['num']=''
for i in data.index:
    data['num'][i]=len(data.at[i, 'path'].split(' > '))

<ipython-input-20-70e08e6591fb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['num'][i]=len(data.at[i, 'path'].split(' > '))


In [21]:
data

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,path,first interaction,last interaction,num
0,1,referral,referral,referral,email,NaN,1,referral > referral > referral > email,referral,email,4
1,1,referral,display,display,email,NaN,1,referral > display > display > email,referral,email,4
2,1,email,display,referral,NaN,NaN,1,email > display > referral,email,referral,3
3,1,referral,referral,email,NaN,NaN,1,referral > referral > email,referral,email,3
4,1,social,referral,NaN,NaN,NaN,1,social > referral,social,referral,2
...,...,...,...,...,...,...,...,...,...,...,...
68119,0,social,display,direct,NaN,NaN,3,social > display > direct,social,direct,3
68120,0,referral,referral,referral,referral,direct,3,referral > referral > referral > referral > di...,referral,direct,5
68121,0,display,referral,social,direct,NaN,3,display > referral > social > direct,display,direct,4
68122,0,email,email,social,direct,NaN,3,email > email > social > direct,email,direct,4


In [22]:
data1=data[data['convert_TF']==1]

In [23]:
data1

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,path,first interaction,last interaction,num
0,1,referral,referral,referral,email,NaN,1,referral > referral > referral > email,referral,email,4
1,1,referral,display,display,email,NaN,1,referral > display > display > email,referral,email,4
2,1,email,display,referral,NaN,NaN,1,email > display > referral,email,referral,3
3,1,referral,referral,email,NaN,NaN,1,referral > referral > email,referral,email,3
4,1,social,referral,NaN,NaN,NaN,1,social > referral,social,referral,2
...,...,...,...,...,...,...,...,...,...,...,...
60648,1,referral,referral,referral,NaN,NaN,3,referral > referral > referral,referral,referral,3
60649,1,display,referral,NaN,NaN,NaN,3,display > referral,display,referral,2
68103,1,organic_search,referral,paid_search,NaN,NaN,3,organic_search > referral > paid_search,organic_search,paid_search,3
68104,1,organic_search,display,NaN,NaN,NaN,3,organic_search > display,organic_search,display,2


In [24]:
channel = []
conversion = []
for i in data1.index:
    for j in data1.at[i, 'path'].split(' > '):
        channel.append(j)
        conversion.append(1/data1['num'][i])
df_linear = pd.DataFrame()
df_linear['Channel'] = channel
df_linear['Conversion'] = conversion



In [33]:
df_linear = df_linear.groupby(['Channel']).sum()
df_linear

,Conversion
Channel,
direct,7.400000
display,2009.466667
email,1275.100000
organic_search,16.150000
paid_search,704.866667
referral,7463.833333
social,2302.183333


### 1.5 average CAC by channel (linear)

In [83]:
linear_cac=[['avg_CAC_linear_direct',0/7.4],
['avg_CAC_linear_display',6000/2009.5],
['avg_CAC_linear_email',6000/1275.1],
['avg_CAC_linear_organic_search',0/16.2],
['avg_CAC_linear_paid_search',6000/704.9],
['avg_CAC_linear_referral',6000/7463.8],
['avg_CAC_linear_social',6000/2302.2]]
linear_CAC=pd.DataFrame(linear_cac, columns=['Channel','CAC'])
linear_CAC

,Channel,CAC
0,avg_CAC_linear_direct,0.000000
1,avg_CAC_linear_display,2.985817
2,avg_CAC_linear_email,4.705513
3,avg_CAC_linear_organic_search,0.000000
4,avg_CAC_linear_paid_search,8.511846
5,avg_CAC_linear_referral,0.803880
6,avg_CAC_linear_social,2.606203


# Part 2

### 2.1 choose first interaction and calculate conversion by tier

In [86]:
df_first=pd.DataFrame()
df_first['Channel']=data['first interaction']
df_first['Attribution']='First interaction'
df_first['Conversion']=data['convert_TF']
df_first['tier']=data['tier']
df_first=df_first.groupby(['Channel','tier']).sum()

<ipython-input-86-b3d1929b1b14>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_first=df_first.groupby(['Channel','tier']).sum()


In [87]:
df_first

Conversion
Channel        tier            
direct         1              1
               2              4
               3              1
display        1            435
               2            726
               3            870
email          1            264
               2            402
               3            618
organic_search 1              1
               2              7
               3              7
paid_search    1            129
               2            211
               3            328
referral       1           1519
               2           2719
               3           3257
social         1            475
               2            823
               3            982